In [1]:
import os
import time
import math
import pickle
from contextlib import nullcontext
import numpy as np
import pandas as pd
import tiktoken
from collections import Counter
import re
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data import DataLoader, Dataset
import inspect
from sklearn.model_selection import train_test_split
import random

In [2]:

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight  # Weight tying

        # Init all weights
        self.apply(self._init_weights)
        # Apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

        # Report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params() / 1e6,))

    def get_num_params(self, non_embedding=True):
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device)  # shape (t)

        # Forward the GPT model itself
        tok_emb = self.transformer.wte(idx)  # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos)  # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # If we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # Inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :])  # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer


In [9]:
import os
import torch
from dataclasses import dataclass

# Define the GPT model and configuration
@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int  # n_embd must be divisible by n_head
    dropout: float
    bias: bool

# Load the saved configuration details
save_dir = r'C:\Users\ajazb\Desktop\AI training\GPTper_saves'
if not os.path.exists(save_dir):
    raise Exception(f"Save directory {save_dir} does not exist!")

# Read the configuration details if needed (e.g., epochs, gradient accumulation steps)
config_path = os.path.join(save_dir, 'training_config_perchin.txt')
with open(config_path, 'r') as f:
    config_details = f.read()
    print(config_details)  # For debugging

# Define the configuration, ensure the values match your saved configuration
config = GPTConfig(
    block_size=64,
    vocab_size=24899,
    n_layer=4,
    n_head=4,
    n_embd=64,  # Ensure n_embd is divisible by n_head
    dropout=0.1,
    bias=True
)

# Initialize the model
model = GPT(config)

# Load the saved model state with map_location to map CUDA tensors to the CPU
model_path = os.path.join(save_dir, 'final_model_perchin.pt')
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Initialize the optimizer with the model parameters
optimizer = torch.optim.Adam(model.parameters())

# Load the saved optimizer state
optimizer_path = os.path.join(save_dir, 'final_optimizer_perchin.pt')
saved_optimizer_state = torch.load(optimizer_path, map_location=torch.device('cpu'))

# Debug prints to compare parameter groups
print(f"Current optimizer param_groups: {len(optimizer.param_groups)}")
print(f"Saved optimizer param_groups: {len(saved_optimizer_state['param_groups'])}")

# Check if the parameter groups match
if len(saved_optimizer_state['param_groups']) != len(optimizer.param_groups):
    print("Parameter groups do not match.")
    # Optionally handle this mismatch, e.g., adjust the model or optimizer definition

# Initialize the optimizer with the model parameters
optimizer = torch.optim.Adam(model.parameters())

# Load the saved optimizer state
optimizer_path = os.path.join(save_dir, 'final_optimizer_perchin.pt')
saved_optimizer_state = torch.load(optimizer_path, map_location=torch.device('cpu'))

# Load the optimizer state manually
for param_group in optimizer.param_groups:
    for p in param_group['params']:
        state = optimizer.state[p]
        if p in saved_optimizer_state['state']:
            state.update(saved_optimizer_state['state'][p])
        else:
            print(f"Parameter {p} not found in saved optimizer state.")

print("Model and optimizer state loaded successfully.")
print(model)


Epochs: 10
Gradient Accumulation Steps: 1

number of parameters: 1.79M
Current optimizer param_groups: 1
Saved optimizer param_groups: 2
Parameter groups do not match.
Parameter Parameter containing:
tensor([[ 0.2393,  0.2208, -0.2748,  ...,  0.3717,  0.1468, -0.2151],
        [ 0.2444,  0.2185, -0.2712,  ...,  0.3731,  0.1413, -0.2155],
        [ 0.2391,  0.2176, -0.2745,  ...,  0.3695,  0.1424, -0.2093],
        ...,
        [ 0.2525,  0.2260, -0.2693,  ...,  0.3687,  0.1382, -0.2086],
        [ 0.2399,  0.2196, -0.2659,  ...,  0.3831,  0.1443, -0.2125],
        [-0.1025,  0.2832,  0.1171,  ..., -0.1070,  0.3294, -0.2589]],
       requires_grad=True) not found in saved optimizer state.
Parameter Parameter containing:
tensor([[ 0.0765, -0.0312, -0.1295,  ..., -0.1225,  0.1956,  0.0979],
        [ 0.0587, -0.0556, -0.0809,  ..., -0.1001,  0.1738,  0.0831],
        [ 0.0564, -0.0677, -0.0654,  ..., -0.0867,  0.1504,  0.0752],
        ...,
        [-0.0960,  0.0220,  0.0866,  ..., -0.016

In [16]:
from transformers import AutoTokenizer

# Load a Persian-specific tokenizer
tokenizer = AutoTokenizer.from_pretrained('bolbolzaban/gpt2-persian')



# Function to tokenize input text and truncate to block size
def tokenize_and_truncate_text(text, block_size):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens) > block_size:
        tokens = tokens[-block_size:]  # Truncate to the last block_size tokens
    return torch.tensor(tokens, dtype=torch.long).unsqueeze(0)  # Add batch dimension

# Function to decode output tokens
def decode_output(predicted_ids):
    predicted_tokens = predicted_ids.tolist()  # Convert tensor to list
    predicted_text = tokenizer.decode(predicted_tokens)
    return predicted_text

# Function to get model output for a single input text
def get_model_output(model, input_text, config, temperature=1.0, top_k=50):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Tokenize and truncate input text
    input_ids = tokenize_and_truncate_text(input_text, config.block_size)
    input_ids = input_ids.to(device)
    model.to(device)

    with torch.no_grad():
        logits, _ = model(input_ids)
        logits = logits[:, -1, :] / temperature  # Apply temperature

        # Top-k sampling
        top_k_logits, top_k_indices = torch.topk(logits, k=top_k, dim=-1)
        probs = F.softmax(top_k_logits, dim=-1)

        # Sample from the distribution
        chosen_idx = torch.multinomial(probs, num_samples=1)
        predicted_token_id = top_k_indices[0, chosen_idx[0]]

        # Decode the predicted token
        predicted_text = decode_output(predicted_token_id)

    return predicted_text

# Function to validate the predicted text
def is_valid_token(token):
    # Implement your logic to check if the token is valid or not
    # This can include checking for known invalid sequences, punctuation, etc.
    invalid_sequences = ['ا ی']
    return token not in invalid_sequences

# Sample input text

input_text = "دست هر کودک ده ساله شهر"
input_text = "یه توپ دارم قلقلیه"
input_text = "دور است کاروان سحر"
input_text = "توانا بود هر که دانا بود"
input_text = "حکیمان پیشین چنین گفته اند"
input_text = "سلام بر تو ای مرد بزرگ"
input_text = "دست هر کودک ده ساله شهر"
input_text = "سلام سلام بچه ها"
input_text = "مرغ سحر ناله سر کن "
input_text = "بنشین که با من هر نظر"


# Define maximum BOM count
max_bom_count = 10

# Accumulate the complete generated text
complete_text = input_text

# Track the last few generated tokens
repetition_threshold = 5
last_few_tokens = []
temperature = 0.7

# Generate text in a loop until the maximum BOM count is reached
bom_count = 0

while bom_count < max_bom_count:
    predicted_text = get_model_output(model, input_text, config, temperature=0.7, top_k=50)

    # Prevent repeating sequences and filter invalid tokens
    if predicted_text.strip() in last_few_tokens or not is_valid_token(predicted_text.strip()):
        temperature *= 1.1  # Increase temperature to promote diversity
        predicted_text = get_model_output(model, input_text, config, temperature=temperature, top_k=50)
    else:
        temperature = max(0.7, temperature * 0.95)  # Gradually decrease temperature back to a minimum

    complete_text += ' ' + predicted_text.strip()  # Ensure proper spacing
    last_few_tokens.append(predicted_text.strip())

    if len(last_few_tokens) > repetition_threshold:
        last_few_tokens.pop(0)

    # Truncate the input text if it exceeds the block size
    input_text += ' ' + predicted_text.strip()
    input_ids = tokenize_and_truncate_text(input_text, config.block_size)
    input_text = tokenizer.decode(input_ids[0].tolist())  # Update input_text to truncated version

    # Add a new line after [EOS]
    if '[EOS]' and '[BOM]' in predicted_text:
        complete_text += '\n'
        bom_count += 1

# Print the whole output after the loop
print(f"Predicted Text:\n{complete_text}\n")


Predicted Text:
بنشین که با من هر نظر بود [BOM]
 گفت که کار گه تو نیست [EOS] [BOM]
 روز ش اندرز کرد جدا [BOM]
 گر چه کنی صاحبخانه ای [EOS] [BOM]
 گر که ندادند ای ریختم بهر ما [BOM]
 چون تو داری سوی من خرده کرد [EOS] [BOM]
 کار تو سالها شد که فکر و کم است [BOM]
 کار من باید رفو ست [EOS] [BOM]
 کار را گفتم دیدی و ابله ی است [BOM]
 مال و گفتی پرده داری که دارو باغبانی بود [EOS] [BOM]


